# ETL Python Notebook from EMR

### Step through code for etl.py script

In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql import Row, functions as F
from pyspark.sql.window import Window

In [6]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']= config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']= config['AWS']['AWS_SECRET_ACCESS_KEY']

#### File paths

In [7]:
input_data = "s3a://udacity-dend/"
output_data = "s3a://cy-udacity-storage/sparkify/"

### Create Spark Session

In [4]:
spark = SparkSession \
        .builder \
        .appName("Sparkify-ETL") \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

### Get song data and extract table data

In [8]:
# get filepath to song data file
song_data = os.path.join(input_data, 'song_data', '*', '*', '*')
    
# read song data file
df = spark.read.json(song_data)

Py4JJavaError: An error occurred while calling o37.json.
: com.amazonaws.services.s3.model.AmazonS3Exception: Status Code: 403, AWS Service: Amazon S3, AWS Request ID: XGJ2MJ80RWH4T78N, AWS Error Code: null, AWS Error Message: Forbidden, S3 Extended Request ID: NQB4x6KQj6tDOXjr291ImAAA6jcfcw4GHdN2gbx0sdfXPDJWg2LHXf5AZaPNmfSYarWRdeviYTM=
	at com.amazonaws.http.AmazonHttpClient.handleErrorResponse(AmazonHttpClient.java:798)
	at com.amazonaws.http.AmazonHttpClient.executeHelper(AmazonHttpClient.java:421)
	at com.amazonaws.http.AmazonHttpClient.execute(AmazonHttpClient.java:232)
	at com.amazonaws.services.s3.AmazonS3Client.invoke(AmazonS3Client.java:3528)
	at com.amazonaws.services.s3.AmazonS3Client.getObjectMetadata(AmazonS3Client.java:976)
	at com.amazonaws.services.s3.AmazonS3Client.getObjectMetadata(AmazonS3Client.java:956)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:892)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.listStatus(S3AFileSystem.java:734)
	at o

In [19]:
# extract columns to create songs table
songs_table = df.select(['song_id', 'title', 'year','duration']).dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# write songs table to parquet files partitioned by year and artist
songs_table.write.partitionBy('year','title').mode('overwrite').parquet(output_data+'songs/songs.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Error sending http request and maximum retry encountered.


In [6]:
# extract columns to create artists table
artists_table = df.select(['artist_id', 'artist_name', 'artist_location', 'artist_latitude','artist_longitude']) \
        .withColumnRenamed('artist_name', 'name') \
        .withColumnRenamed('artist_location', 'location') \
        .withColumnRenamed('artist_latitude', 'latitude') \
        .withColumnRenamed('artist_longitude', 'longitude') \
        .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# write artists table to parquet files
artists_table.write.mode('overwrite').parquet(output_data+'artists/artists.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Get Log data and extract table data

In [5]:
# get filepath to log data file
log_data = os.path.join(input_data, 'log_data','*','*')

# read log data file
log_df = spark.read.json(log_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# filter by actions for song plays
log_df = log_df.filter(log_df['page'] == 'NextSong') 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# extract columns for users table 
users_table = log_df.select(['userId', 'firstName', 'lastName', 'gender', 'level', \
                         F.row_number().over(Window.partitionBy('userId').orderBy(F.desc('ts'))).alias('rowNum')]) 
users_table = users_table.select(['userId', 'firstName', 'lastName', 'gender', 'level']) \
                        .filter(users_table.rowNum == 1)  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# write users table to parquet files
users_table.write.mode('overwrite').parquet(output_data+'users/users.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# create timestamp column from original timestamp column
get_timestamp = udf(lambda x: str(int(int(x)/1000)))
log_df = log_df.withColumn('timestamp', get_timestamp(log_df.ts))

# create datetime column from original timestamp column
get_datetime = udf(lambda x: str(datetime.fromtimestamp(int(x) / 1000)))
log_df = log_df.withColumn('start_time', get_datetime(log_df.ts))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# extract columns to create time table
time_table = log_df.select('start_time') \
                .withColumn('hour', F.hour('start_time')) \
                .withColumn('day', F.dayofmonth('start_time')) \
                .withColumn('week', F.weekofyear('start_time')) \
                .withColumn('month', F.month('start_time')) \
                .withColumn('year', F.year('start_time')) \
                .withColumn('weekday', F.when(F.dayofweek('start_time') < 5, 1).otherwise(0)) \
                .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# write time table to parquet files partitioned by year and month
time_table.write.partitionBy('year','month').mode('overwrite').parquet(output_data+'time/time.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Create Songplays table

In [12]:
# read in song data to use for songplays table
song_data = os.path.join(input_data, 'song_data', '*', '*', '*')
song_df = spark.read.json(song_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# extract columns from joined song and log datasets to create songplays table 
log_song_table = log_df.join(song_df, (log_df.song == song_df.title)\
                             & (log_df.artist == song_df.artist_name) \
                             & (log_df.length == song_df.duration), how='inner')

songplays_table = log_song_table.select([F.monotonically_increasing_id().alias('songplay_id'),'start_time', 'userId','level','song_id','artist_id', 'sessionId', 'location', 'userAgent']) \
                                .withColumn('month', F.month('start_time')) \
                                .withColumn('year', F.year('start_time')) \
                                .withColumnRenamed('userId', 'user_id') \
                                .withColumnRenamed('userAgent', 'user_agent') \
                                .withColumnRenamed('sessionId', 'session_id')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# write songplays table to parquet files partitioned by year and month
songplays_table.write.partitionBy('year','month').mode('overwrite').parquet(output_data+'songplays/songplays.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Terminate spark session

In [9]:
spark.stop()